In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

import warnings
warnings.filterwarnings("ignore")
import random

import hgd_dtw

## Heterogeneous Gower Distance
* IMPLEMENTATION OF GOWER DISTANCE CONSIDERING THE NATURE OF EACH VARIABLE, WITH CORRESPONDING NORMALIZATION AND RESCALING

## Features

In [ ]:
folders = ["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10"
]

norm = "minMax_per_patient"

keys = ['AFA', 'AFE', 'AFO', 'AFS', 'AMG', 'CAR',
       'CF1', 'CF2', 'CF3', 'CFO', 'CPS', 'GCC', 'GLI', 'IBL', 'LIN', 'LIP',
       'MAC', 'MON', 'NTI', 'OTR', 'OXA', 'Otros', 'PEN', 'POL', 'QUI', 'SUL',
       'TMS', 'TTC', 'horas_VM', 'contMask', 'numberOfPatients', 'numberOfPatientsMR',
       'neighbor_CAR', 'neighbor_IBL', 'neighbor_AFO', 'neighbor_OXA',
       'neighbor_QUI', 'neighbor_PEN', 'neighbor_AFA', 'neighbor_CF3',
       'neighbor_GLI', 'neighbor_CPS', 'neighbor_TMS', 'neighbor_LIN',
       'neighbor_NTI', 'neighbor_MAC', 'neighbor_OTR', 'neighbor_AMG',
       'neighbor_AFE', 'neighbor_POL', 'neighbor_CF1', 'neighbor_GCC',
       'neighbor_LIP', 'neighbor_AFS', 'neighbor_MON', 'neighbor_CFO',
       'neighbor_Otros', 'neighbor_TTC', 'neighbor_CF2', 'neighbor_SUL',
       'cambio_postural', 'insulina', 'nutr_art', 'sedacion', 'relajacion',
       'n_transf', 'farm_vasoactivos', 'dosis_nems', 'hoursTraqueo',
       'hoursConUlcera', 'hoursConHemo', 'C01 VC acceso periférico 1',
       'C01 VC acceso periférico 2', 'C02 Vía central - YD',
       'C02 Vía central - SD', 'C02 Vía central - SI', 'C02 Vía central - FD',
       'C02 Vía central - YI', 'C02 Vía central - FI', 'numberOfCatheters',
       'pc_acinet', 'pc_enteroc', 'pc_pseud', 'pc_staph', 'pc_no_germ']

binary = ['AFA', 'AFE', 'AFO', 'AFS', 'AMG', 'CAR',
       'CF1', 'CF2', 'CF3', 'CFO', 'CPS', 'GCC', 'GLI', 'IBL', 'LIN', 'LIP',
       'MAC', 'MON', 'NTI', 'OTR', 'OXA', 'Otros', 'PEN', 'POL', 'QUI', 'SUL',
       'TMS', 'TTC', 
       'cambio_postural', 'insulina', 'nutr_art', 'sedacion', 'farm_vasoactivos',
       'pc_acinet', 'pc_enteroc', 'pc_pseud', 'pc_staph', 'pc_no_germ'
       ]


continuous =  [variable for variable in keys if variable not in binary]

## Considering by time step
* The relationships are assessed independently at each time step

In [ ]:
folders = ["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10"
]

In [ ]:
for c in range(len(folders)):
    
    numberOfTimeStep = 14
    ########################

    ####### BEGIN CODE ===>>
    # Load data
    X_train = np.load("../../../DATA/" + folders[c] + "/X_train_tensor_"+norm+".npy")
    
    # Reshape data by features --> PxTxF to FxTxP
    X_train = hgd_dtw.reshape_patients_by_features(X_train, keys, numberOfTimeStep)
    print("X_train shape after reshape:", X_train.shape)

    for nt in range(numberOfTimeStep):
        # Generate a correlation matrix by matrix
        x = X_train[:, nt, :] 
        
        mat_by_time_step = np.zeros((x.shape[0], x.shape[0]))

        for i in range(x.shape[0]):
            for j in range(x.shape[0]):
                mat_by_time_step[i, j] = hgd_dtw.hgd_distance(x[i], x[j], keys[i], keys[j], binary, continuous)

        S = hgd_dtw.diagonal_to_zero(mat_by_time_step)

        pd.DataFrame(S, columns=keys, index=keys).to_csv("./estimatedGraphs/dtw-hgd/" + folders[c] + "/adj_X_train_TS_" + str(nt) + "_" + norm + ".csv", index=False)
